In [1]:
%matplotlib inline
import os
import hugs
import pandas as pd
import matplotlib.pyplot as plt
import webbrowser
from astropy.table import Table
plt.style.use('jpg')
from toolbox.cats import crossmatch
from toolbox.cosmo import Cosmology
from toolbox.astro import angsep
cosmo = Cosmology()

In [2]:
path = os.path.join(os.environ.get('HUGS_PIPE_IO'), 'patches-run-20170208-223945')
butler = hugs.datasets.CatButler(path)
out_fn = os.path.join(path, 'patch-combined-cat.csv')
out_fn_no_dup = os.path.join(path, 'patch-combined-cat-no-duplicates.csv')

In [3]:
local_io = os.environ.get('LOCAL_DATA')
oldcat = pd.read_csv(os.path.join(local_io, 'hsc/stamps/candy/20170130-143629/candy.csv'))

In [5]:
if False:
    cat = butler.combine_patch_cats()
    cat.to_csv(out_fn, index=False)
    hugs.datasets.hugged.remove_duplicates(cat)
    cat.to_csv(out_fn_no_dup, index=False)
else:
    cat = pd.read_csv(out_fn_no_dup)
cat['g-i'] = cat['MAG_APER_0(g)'] - cat['MAG_APER_0(i)']
cat['g-r'] = cat['MAG_APER_0(g)'] - cat['MAG_APER_0(r)']

In [153]:
m, b = 0.6724944691862157, -0.3313339303786671
color_line = lambda _x: m*_x + b

In [48]:
need_to_recover = [0, 101, 102, 103, 109, 115, 116, 120, 121, 
                   125, 126, 130, 134, 135, 137, 139, 140,
                   142, 146, 153, 159, 162, 164, 171, 177, 
                   182, 184, 187, 190, 193, 196, 199, 202, 
                   209, 212, 24, 25, 26, 27, 28, 38, 39, 4, 63, 69]

In [70]:
nice_to_recover = [104, 136, 144, 15, 160, 169, 174, 175, 2, 214, 3, 31, 5, 50, 58, 74, 97]

In [253]:
ra, dec = oldcat.loc[196, ['ra', 'dec']]
def hsc_map(ra, dec):
    url = 'https://hscdata.mtk.nao.ac.jp/hsc_ssp/dr1/s16a/hscMap/'
    url += '#%7B%22tract%22%3A%22s16a_wide%2F9560%22%2C%22view%22%'
    url += '3A%7B%22center%22%3A%7B%22lat%22%3A'+str(ra)+'%2C%22lng%22%3A'
    url += str(dec)+'%7D%2C%22zoom%22%3A18%7D%2C%22colorFilter%22%3A%22SDSS%'
    url += '20True%20Color%22%2C%22colorFilter%2Fparams%2FSDSS%20True%20Color'
    url += '-1%22%3A%7B%22bias%22%3A0%2C%22log_a%22%3A10.819778284410283%2C%22format'
    url += '%22%3A%22png%22%2C%22filters%22%3A%5B%22I%22%2C%22R%22%2C%22G%22%5D%7D%7D'
    webbrowser.open(url, new=1);

In [154]:
old_tab = Table.from_pandas(oldcat.loc[nice_to_recover].copy())
new_tab = Table.from_pandas(cat.copy())
old_matched, new_matched = crossmatch(old_tab, new_tab, maxsep=1.0)

In [397]:
cut = cat['mu_aper_0(g)'] > 24.0
#cut &= cat['r_e(i)'] < 100
#cut &= cat['r_e(i)'] > 3.0
cut &= cat['FLUX_RADIUS(i)'] > 2.5/0.168
cut &= np.abs(cat['r_e(i)']-cat['r_e(g)'])<5.0
cut &= np.abs(cat['dr_e(i)']) < 5.0
cut &= np.abs(cat['dmu(i)']) < 1.0
cut &= np.abs(cat['dmu(g)']) < 1.0
cut &= np.abs(cat['dR0']) < 15
#cut &= cat['n'] < 1.4
cut &= cat['n'] > 0.01
cut &= cat['ell']<0.70

color_cut = (cat['g-r'] > color_line(cat['g-i']))
color_cut &= cat['g-i'] < 1.9
color_cut &= cat['g-i'] > 0.
color_cut &= cat['g-r'] < 1.5
color_cut &= cat['g-r'] > 0.
cut &= color_cut

sample = cat[cut].copy()
#hugs.datasets.hugged.remove_duplicates(sample)

idx = need_to_recover
old_tab = Table.from_pandas(oldcat.loc[idx].copy())

sample_table = Table.from_pandas(sample.copy())
old_matched, new_matched = crossmatch(old_tab, sample_table, maxsep=0.7)
print len(idx), len(new_matched), len(sample)

45 39 604


In [391]:
missing = []
for i, (ra, dec) in enumerate(old_tab['ra', 'dec']):
    if angsep(ra, dec, old_matched['ra'], old_matched['dec']).min()>0.7:
        missing.append(i)

In [398]:
#old_tab[missing]['ra', 'dec', 'mu_0(g)', 'FLUX_RADIUS(i)'].as_array()

In [372]:
for ra, dec in old_tab[missing]['ra', 'dec']:
    hsc_map(ra, dec)

In [77]:
if False:
    for i, (ra, dec) in enumerate(old_tab['ra', 'dec']):
        print need_to_recover[i], angsep(ra, dec, new_tab['ra'], new_tab['dec']).min()
    tract, patch, ra, dec, ximage, yimage = oldcat.loc[196, ['tract', 'patch', 'ra', 'dec', 'X_IMAGE', 'Y_IMAGE']]
    miss_cat = butler.get_patch_cat(tract, patch, kind='all')

In [395]:
sample.to_csv('/Users/protostar/Desktop/sample-aggresive-cuts.csv', index=False)